<a href="https://colab.research.google.com/github/SimeonHristov99/CodeEveryDay/blob/main/Dogs_vs_Cats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import os
import zipfile
from glob import glob

import tensorflow as tf
print(tf.__version__)

import numpy as np
import matplotlib.pyplot as plt

2.7.0


In [5]:
config = {
    'kaggle_dir': '../gdrive/MyDrive/kaggle',
}

In [13]:
os.chdir('/content')
if not os.path.isdir('data'):
  kaggle_dir = config.get('kaggle_dir', None)

  from google.colab import drive
  drive.mount('/gdrive')
  
  assert kaggle_dir is not None and os.path.isdir(kaggle_dir), 'Kaggle directory not found!'
  assert os.path.isfile(f'{kaggle_dir}/kaggle.json'), 'Kaggle API token not found!'

  !pip install -q kaggle
  os.environ['KAGGLE_CONFIG_DIR'] = kaggle_dir

  # <insert kaggle API command here>
  # <example
  # !kaggle datasets download -d humansintheloop/semantic-segmentation-of-aerial-imagery
  # >
  !kaggle competitions download -c dogs-vs-cats -p /content/data/
  assert os.path.isdir('data'), 'ERROR: Could not download the dataset!'
  print('Successfully downloaded the dataset!')
else:
  print('Dataset already downloaded.')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
 97% 529M/543M [00:04<00:00, 125MB/s]
100% 543M/543M [00:04<00:00, 129MB/s]
  0% 0.00/86.8k [00:00<?, ?B/s]
100% 86.8k/86.8k [00:00<00:00, 74.3MB/s]
 99% 269M/271M [00:03<00:00, 92.4MB/s]
100% 271M/271M [00:03<00:00, 80.5MB/s]
Successfully downloaded the dataset!


In [14]:
with zipfile.ZipFile("/content/data/test1.zip","r") as zip_ref:
    zip_ref.extractall("/content/data/test1")

with zipfile.ZipFile("/content/data/train.zip","r") as zip_ref:
    zip_ref.extractall("/content/data/train")

In [28]:
len(glob('/content/data/train/train/dog*'))

12500

In [29]:
len(glob('/content/data/train/train/cat*'))

12500

In [30]:
len(glob('/content/data/train/train/*'))

25000

Classes are balanced.

In [44]:
def get_x_y(path):
  X = np.array(os.listdir(path))
  y = np.array([ x.split('.')[0].capitalize() for x in X ])
  return X, y

In [45]:
X, y = get_x_y('/content/data/train/train')

In [46]:
X

array(['dog.3632.jpg', 'dog.7775.jpg', 'cat.225.jpg', ..., 'cat.1950.jpg',
       'dog.3212.jpg', 'cat.5132.jpg'], dtype='<U13')

In [47]:
y

array(['Dog', 'Dog', 'Cat', ..., 'Cat', 'Dog', 'Cat'], dtype='<U3')

Next: Create the tf.data pipeline.